# Sistemas Inteligentes 2023/2024

## Avaliação Contínua 2: Um par de heurísticas para o Automóvel no Labirinto

### Entrega: 27 de Março às 23:59

<img src="Figuras\roadtrip.gif" alt="Drawing" style="width: 150px;"/>



## Introdução

Considere o problema do Automóvel no Labirinto que foi tema da avaliação contínua 1 e que relembramos aqui.

<img src="Figuras\car_smoke.gif" alt="Drawing" style="width: 180px;"/>

Imagine um automóvel que deseja sair de um labirinto discreto. O automóvel (agente) ocupa uma célula e é direcional podendo estar orientado para Norte (N), Sul (S), Este (E) ou Oeste (O). Com uma única ação o agente pode mudar de orientação ou avançar com uma velocidade ajustável. 

O objetivo do automóvel é, partindo parado (com velocidade 0) de uma posição marcada e com uma determinada orientação, encontrar um plano para estacionar no quadrado de saída utilizando o menor número de ações possiveis. Estacionar o automóvel significa chegar à posição objetivo com velocidade zero.

## Objetivos
<img src="Figuras\driving.gif" alt="Drawing" style="width: 150px;"/>
Dada a implementação em python da modelização do Carro no Labirinto como um problema de procura num grafo, de acordo com o módulo `searchPlus.py`, deve completá-la implementando duas funções heurísticas que estimam o menor custo até ao objectivo: as funções `minAeTs` e `minRotacoes`, as quais correspondem às soluções de duas versões "relaxadas" do problema original. 

## Modelização em Python

<img src="Figuras\VW181.jpg" alt="Drawing" style="width: 230px;"/>

A seguir apresentamos a implementação da classe `Labirinto` para a modelização do problema como um grafo de estados, que utilizámos como referência para a realização dos testes da Avaliação Contínua 1.

Nesta modelização usámos um triplo (tuplo com 3 campos) para guardar a informação que muda com as acções: 
* a posição: (linha,coluna),
* a orientação que pode ser {'N', 'S', 'O', 'E'},
* e a velocidade do carro.

Como informação partilhada, criámos os seguintes atributos:
* `dim`: a dimensão do labirinto (Ls,Cs) em Ls e Cs representam o número de linhas e de colunas respectivamente.
* `vmax`: a indicação da velocidade máxima
* `obstacles`: o conjunto de coordenadas das paredes

In [1]:
from searchPlus import *

line1 = "= = = = = = =\n"
line2 = "= x . . . . =\n"
line3 = "= . . . = . =\n"
line4 = "= . . . = . =\n"
line5 = "= = = . = . =\n"
line6 = "= ^ . . . . =\n"
line7 = "= = = = = = =\n"
grelha = line1 + line2 + line3 + line4 + line5 + line6 + line7

class Labirinto(Problem):

    def process_txt(self, grid):
        data = {'walls': set()}
        lines = grid.split('\n')
        x = 0
        for row in lines:
            y = 0
            for col in row:
                if col == '=':
                    data['walls'].add((x,y))
                elif col == 'x':
                    data['exit'] = (x,y)
                elif col == '^':
                    data['car'] = (x,y)
                    data['direction'] = 'N'
                elif col == '>':
                    data['car'] = (x,y)
                    data['direction'] = 'E'
                elif col == '<':
                    data['car'] = (x,y)
                    data['direction'] = 'O'
                elif col == 'v':
                    data['car'] = (x,y)
                    data['direction'] = 'S'
                if col != " ":
                    y += 1
            x += 1
        data['dim'] = (len(lines)-1,(len(lines[0])+1)//2)
        return data

    
    directions = {"N":(-1, 0), "E":(0, +1), "S":(+1, 0), "O":(0, -1)}  # ortogonals

    
    def __init__(self, LabInicial=grelha, vmax=3):
        initialStatus = self.process_txt(LabInicial) # process txt and convert to a dictionary
        
        self.initial = (initialStatus['car'], initialStatus['direction'], 0)  # tuple with car position, direction and velocity
        # self.direction = initialStatus['direction'] # car direction (N, S, E, W)
        self.goal = initialStatus['exit'] # goal position
        self.obstacles = initialStatus['walls'] # walls positions
        self.dim = initialStatus['dim'] # maze dimension (do not need to be squared)
        self.vmax = vmax

        
    def actions (self, state):
        """L = rotate the car 90º left; R = rotate the car 90º right; 
         M = move forward and add 1 unit to the velocity; B = break and decrease 1 unit to the velocity"""
        x, y = state[0] # car position
        d = state[1] # direction
        v = state[2] # velocity
        action_list = []
        
        # if the car is facing an obstacle, rotate the car 90º 
        if v == 0:
            action_list.extend(['E','D'])
            
        # if the car is facing no obstacles and it can move the number of spaces given by velocity+1
        pos = [(x+(a+1)*self.directions[d][0], y+(a+1)*self.directions[d][1]) for a in range(0,v+1)]
        if all([a not in self.obstacles for a in pos]) \
        and 0 < x+(v+1)*self.directions[d][0] < self.dim[0] \
        and 0 < y+(v+1)*self.directions[d][1] < self.dim[1] :
            action_list.append('A')
            
        # if velocity is higher than 0, break is a possible action if it can move the number of spaces given by velocity-1
        pos = [(x+(a-1)*self.directions[d][0], y+(a-1)*self.directions[d][1]) for a in range(0,v+1)]
        if v > 0 and all([a not in self.obstacles for a in pos]): 
            action_list.append('T')
            
        return sorted(action_list)

    
    def result (self, state, action):
        x, y = state[0]
        d = state[1]
        v = state[2]
        if action == 'T' and v > 0:
            v -= 1
            pos = (x+v*self.directions[d][0], y+v*self.directions[d][1])
            new_dir = d
        if action == 'E':
            new_dir = list(p.directions.keys())[list(p.directions.keys()).index(d)-1]
            pos = (x, y)
        if action == 'D':
            new_dir = list(p.directions.keys())[list(p.directions.keys()).index(d)+1 if d!='O' else 0]
            pos = (x, y)
        if action == 'A':
            if v < self.vmax:
                v += 1
            pos = (x+v*self.directions[d][0], y+v*self.directions[d][1])
            new_dir = d
        return (pos,new_dir,v)

    
    def goal_test (self, state):
        return state[0] == self.goal and state[2] == 0

    
    def display (self, state):
        """Devolve a grelha em modo txt"""
        output = ""
        for i in range(self.dim[0]):
            for j in range(self.dim[1]):
                if state[0] == (i,j) and state[1] == 'N':
                    ch = "^"
                elif state[0] == (i,j) and state[1] == 'S':
                    ch = "v"
                elif state[0] == (i,j) and state[1] == 'E':
                    ch = ">"
                elif state[0] == (i,j) and state[1] == 'O':
                    ch = "<"
                elif self.goal == (i,j):
                    ch = "x"
                elif (i,j) in self.obstacles:
                    ch = "="
                else:
                    ch = "."
                output += ch + " "
            output += "\n"
        return output

    
    def executa(self, state, actions_list, verbose=False):
        """Executa uma sequência de acções a partir do estado devolvendo o triplo formado pelo estado, 
        pelo custo acumulado e pelo booleano que indica se o objectivo foi ou não atingido. Se o objectivo 
        for atingido antes da sequência ser atingida, devolve-se o estado e o custo corrente.
        Há o modo verboso e o não verboso, por defeito."""
        cost = 0
        for a in actions_list:
            seg = self.result(state,a)
            cost = self.path_cost(cost,state,a,seg)
            state = seg
            obj = self.goal_test(state)
            if verbose:
                print('Ação:', a)
                print(self.display(state),end='')
                print('Custo Total:',cost)
                print('Atingido o objectivo?', obj)
                print()
            if obj:
                break
        return (state, cost, obj)
         
    def minRotacoes(self,node):
        pass
    
    def minAeTs(self,node):
        pass
    


Podemos confirmar que está a funcionar:

In [2]:
line1 = "= = = = = = = = = = = = = = = = = = = =\n"
line2 = "= x . . . . . . . . . . . . . . . . . =\n"
line3 = "= . . . . . . . . . . . . . . . . . > =\n"
line4 = "= = = = = = = = = = = = = = = = = = = =\n"
grelha2 = line1 + line2 + line3 + line4
p = Labirinto(grelha2,vmax=3)
resultado = breadth_first_graph_search(p)
if resultado:
    print("Solução Larg-prim (grafo) com custo", str(resultado.path_cost)+":")
    print(resultado.solution())
else:
    print("Sem solução!")

Solução Larg-prim (grafo) com custo 13:
['E', 'A', 'T', 'E', 'A', 'A', 'A', 'A', 'T', 'A', 'T', 'T', 'T']


## O método `minRotacoes`
Esta heurística corresponde à solução em termos do número mínimo de acções que são a solução de uma versão relaxada do problema original em que as acções são apenas as duas rotações, ignorando os deslocamentos. Queremos estimar o número mínimo de rotações que o carro precisa de fazer para atingir a solução, ignorando que há paredes.

* Se o carro satisfizer o objectivo, o valor desta heurística será 0.
* Mas se estiver na posição do objectivo e a sua velocidade for diferente de 0, o seu valor será de 2. Precisará sempre de inverter o sentido de marcha.
* Um carro na mesma linha do objectivo e virado para o objectivo já não precisará de fazer rotações, mas se estiver virado em sentido contrário terá que fazer pelo menos 2 rotações, para inverter o sentido de marcha.
* O resto das situações ficará por vossa conta.

### Exemplos Abertos de Teste
<img src="Figuras\vulcanizing.gif" alt="Drawing" style="width: 300px;"/>

#### Exemplo 1
Neste exemplo, o carro está na mesma coluna e virado para o objectivo: só terá de saltar para o objectivo (na versão relaxada salta sem custo)
```python
p = Labirinto(grelha)
print(p.display(p.initial))
print(p.minRotacoes(Node(p.initial)))
= = = = = = = 
= x . . . . = 
= . . . = . = 
= . . . = . = 
= = = . = . = 
= ^ . . . . = 
= = = = = = = 

0
```

#### EXemplo 2
Neste exemplo, o carro está na mesma linha mas mal orientado, i.e., não virado para o objectivo: só terá de rodar para a direita para apontar ao objecto e depois salta sem custo. Apenas necessita de uma rotação.
```python
line1 = "= = = = = = = = = =\n"
line2 = "= x . v . . . . . =\n"
line3 = "= . . . = . . . . =\n"
line4 = "= . . . = . = . . =\n"
line5 = "= = = . = . = . . =\n"
line6 = "= . . . . . . . = =\n"
line7 = "= = = = = = = = = =\n"
grelha2 = line1 + line2 + line3 + line4 + line5 + line6 + line7
p2 = Labirinto(grelha2)
print(p2.display(p2.initial))
print(p2.minRotacoes(Node(p2.initial)))
= = = = = = = = = = 
= x . v . . . . . = 
= . . . = . . . . = 
= . . . = . = . . = 
= = = . = . = . . = 
= . . . . . . . = = 
= = = = = = = = = = 

1
```

#### Exemplo 3
Neste exemplo, o carro noutra linha e mal orientado, i.e., não virado para o objectivo: só terá de rodar para a direita para apontar ao objecto e depois voltar a rodar.
```python
line1 = "= = = = = = = = = =\n"
line2 = "= x . . . . . . . =\n"
line3 = "= . . v = . . . . =\n"
line4 = "= . . . = . = . . =\n"
line5 = "= = = . = . = . . =\n"
line6 = "= . . . . . . . = =\n"
line7 = "= = = = = = = = = =\n"
grelha2 = line1 + line2 + line3 + line4 + line5 + line6 + line7
p2 = Labirinto(grelha2)
print(p2.display(p2.initial))
print(p2.minRotacoes(Node(p2.initial)))
= = = = = = = = = = 
= x . . . . . . . = 
= . . v = . . . . = 
= . . . = . = . . = 
= = = . = . = . . = 
= . . . . . . . = = 
= = = = = = = = = = 

2
```

#### Exemplo 4
Aqui partimos do exemplo por omissão, mas mudamos o o carro para cima do objectivo mas com velocidade 2 .
```python
p = Labirinto(grelha)
l,c=p.goal
p.initial=((l,c),'N',2)
print(p.display(p.initial))
print(p.minRotacoes(Node(p.initial)))
= = = = = = = 
= ^ . . . . = 
= . . . = . = 
= . . . = . = 
= = = . = . = 
= . . . . . = 
= = = = = = = 

2
```

#### Exemplo 5
Neste em que usamos a grelha original (por defeito), vamos aplicar o Greedy.
```python
p = Labirinto(grelha)
res = greedy_best_first_graph_search(p,p.minRotacoes)
print(res.solution())
print(res.path_cost)
['D', 'A', 'T', 'A', 'T', 'E', 'A', 'T', 'A', 'T', 'A', 'T', 'E', 'A', 'T', 'A', 'T', 'D', 'A', 'T']
20
```

#### Exemplo 6
Neste em que usamos a grelha original (por defeito), vamos aplicar o A*.
```python
p = Labirinto(grelha)
res = greedy_best_first_graph_search(p,p.minRotacoes)
print(res.solution())
print(res.path_cost)
['D', 'A', 'T', 'A', 'T', 'E', 'A', 'A', 'T', 'T', 'E', 'A', 'T', 'A', 'T']
15
```

## O método `minAeTs`
Esta heurística corresponde à solução em termos do número mínimo de acções `A`s e `T`s que são a solução da versão relaxada do problema original em que não são necessárias rotações e em que se ignoram os obstáculos. Iremos ter só as acções `A`e `T`. Consideramos que o carro não tem orientação e tem apenas que se dirigir da forma mínima para a coluna e linha do objectivo, parando nessa coluna e linha com velocidade 0.

No fundo este problema devolve o número mínimo de acelarações e travagens necessárias para chegar ao objectivo com velocidade 0, ignorando totalmente as rotações e os obstáculos, por isso é uma versão a que se chama de relaxada. Para chegar ao objectivo com velocidade 0 terá de galgar a diferenças de linhas de modo a chegar à mesma linha com velocidade 0 e depois galgar o nº mínimo de colunas necessárias para chegar ao objectivo com velocidade 0.

### Testes Abertos
<img src="Figuras\car_buster2.gif" alt="Drawing" style="width: 300px;"/>

#### Exemplo 1
Neste exemplo, o carro está na mesma coluna tem de se deslocar 4 linhas até ao objectivo, ignorando a parede. Para galgar 4 linhas terminando com velocidade 0, considerando que a velocidade máxima é 3, terá no mínimo de fazer 4 movimentos: AATT
```python
p = Labirinto(grelha)
print(p.display(p.initial))
print(p.minAeTs(Node(p.initial)))
= = = = = = = 
= x . . . . = 
= . . . = . = 
= . . . = . = 
= = = . = . = 
= ^ . . . . = 
= = = = = = = 

4
```

#### Exemplo 2
Neste exemplo, o carro não está nem na mesma coluna nem na mesma linha e terá que se deslocar 1 casa para ir para a linha do objectivo e 2 casas para a coluna do objectivo. Para galgar essa casa acabando com velocidade 0 e depois galgar 2 casas acabando com velocidade 0 terá no mínimo de fazer 2 movimentos para cima (AT) e 4 movimentos para a esquerda (ATAT). O resultado são 6 acções mínimas. Notem que tivemos em consideração que a velocidade máxima é de 3, por defeito.
```python
line1 = "= = = = = = = = = =\n"
line2 = "= x . . . . . . . =\n"
line3 = "= . . v = . . . . =\n"
line4 = "= . . . = . = . . =\n"
line5 = "= = = . = . = . . =\n"
line6 = "= . . . . . . . = =\n"
line7 = "= = = = = = = = = =\n"
grelha2 = line1 + line2 + line3 + line4 + line5 + line6 + line7
p2 = Labirinto(grelha2)
print(p2.display(p2.initial))
print(p2.minAeTs(Node(p2.initial)))
= = = = = = = = = = 
= x . . . . . . . = 
= . . v = . . . . = 
= . . . = . = . . = 
= = = . = . = . . = 
= . . . . . . . = = 
= = = = = = = = = = 

6
```

#### Exemplo 3
Neste exemplo, o carro tem de galgar 6 colunas 4 linhas para chegar ao objectivo. Como a velocidade máxima é 1, ele pode galgar 6 espaços de forma mínima em 7 movimentos: AAAAAAT e galgar 4 espaços em 5 movimentos: AAAAT. No total teremos 12 movimentos mínimos.
```python
line1 = "= = = = = = = = = = =\n"
line2 = "= x . . . . . . . . =\n"
line3 = "= . . . = . . . . . =\n"
line4 = "= . . . = . = . . . =\n"
line5 = "= = = . = . = . . . =\n"
line6 = "= . . . . . . > = . =\n"
line7 = "= = = = = = = = = . =\n"
grelha2 = line1 + line2 + line3 + line4 + line5 + line6 + line7
p2 = Labirinto(grelha2,vmax=1)
print(p2.display(p2.initial))
print(p2.minAeTs(Node(p2.initial)))
= = = = = = = = = = = 
= x . . . . . . . . = 
= . . . = . . . . . = 
= . . . = . = . . . = 
= = = . = . = . . . = 
= . . . . . . > = . = 
= = = = = = = = = . = 

12
```

#### Exemplo 4
Neste exemplo, o carro tem de galgar 2 linhas e 5 colunas para chegar ao objectivo. Como a velocidade máxima é 2, ele pode galgar 6 espaços de forma mínima em 5 movimentos: AAATT e galgar 2 espaços em 4 movimentos: AATT. No total teremos 9 movimentos mínimos.
```python
line1 = "= = = = = = = = = = =\n"
line2 = "= . . . . . . . . . =\n"
line3 = "= . > . = . . . . . =\n"
line4 = "= . . . = . = . . . =\n"
line5 = "= = = . = . = . x . =\n"
line6 = "= . . . . . . . = . =\n"
line7 = "= = = = = = = = = = =\n"
grelha2 = line1 + line2 + line3 + line4 + line5 + line6 + line7
p2 = Labirinto(grelha2,vmax=2)
print(p2.display(p2.initial))
print(p2.minAeTs(Node(p2.initial)))
= = = = = = = = = = = 
= . . . . . . . . . = 
= . > . = . . . . . = 
= . . . = . = . . . = 
= = = . = . = . x . = 
= . . . . . . . = . = 
= = = = = = = = = = = 

9
```

Testemos agora as procuras heurísticas

#### Exemplo 5
```python
p = Labirinto(grelha)
res_astar = astar_search(p,p.minAeTs)
print(res_astar.solution())
print(res_astar.path_cost)
['D', 'A', 'T', 'A', 'T', 'E', 'A', 'A', 'T', 'T', 'E', 'A', 'T', 'A', 'T']
15
```

#### Exemplo 6
```python
p = Labirinto(grelha)
res_astar = greedy_best_first_graph_search(p,p.minAeTs)
print(res_astar.solution())
print(res_astar.path_cost)
['D', 'A', 'T', 'A', 'T', 'E', 'A', 'A', 'T', 'T', 'E', 'A', 'T', 'A', 'T']
15
```

## Submissão

<img src="Figuras\espelho_retrovisor.gif" alt="Drawing" style="width: 250px;"/>

#### Quizz

Cada grupo deve completar a implementação das duas funções heurísticas: `minAeTs` e `minRotacoes`, métodos da classe `Labirinto` e testá-las no link do quizz **Avaliação Contínua 2** que está na página da disciplina, introduzindo aí o vosso código. Os vários elementos do grupo podem desenvolver, submeter e avaliar o código várias vezes, sendo a submissão com melhor nota a que será considerada.

Esse quizz é constituído por uma única pergunta, que cobre a implementação dos dois métodos e que serão avaliados através de um conjunto de testes automáticos visíveis e outros invisíveis, valendo um total de $1.0$ valores ($0.5$ para cada uma delas).

Os testes visíveis valem $6\space(3+3)$ em $20$, enquanto que os testes invisíveis valem $14\space(7+7)$ em $20$.

#### Ficheiro Pyhton

Simultaneamente, é necessário submeter o script Python que contém todo o código submetido no quizz. Só se pretende uma submissão por grupo. Esse ficheiro deve chamar-se **SI-proj2-XX.py**, em que substituem XX pelo número do grupo.

<span style="color:red">ATENÇÂO: Se não submeterem o ficheiro python ou não estiverem inscritos num grupo, não terão nota nesta avaliação contínua.</span>.

Dúvidas: Quasiquer dúvidas que tenham a ver com a interpretação do enunciado, devem enviar para o fórum da disciplina. Qualquer outra dúvida enviem para pub@di.fc.ul.py.

<img src="Figuras\the_end.gif" alt="Drawing" style="width: 200px;"/>